# Tabla de Contenidos

1. [Introducción](#first-bullet)
2. [Configuración del Entorno](#second-bullet)
3. [Creación de Secretos Estáticos](#creación-de-secretos-estáticos)
   1. [Binarios (Base64 JKS + Password)](#binarios)
   2. [Secretos como Texto](#secretos-como-texto)
   3. [Claves de Criptografía](#crypto)
4. [Operaciones de Criptografía usando la Transit Secret Engine](#transit)

# Introducción <a class="anchor" id="first-bullet"></a>
El objetivo de este notebook es el de ilustrar algunas opciones de configuración de Vault, Vault Secret Operator y Vault Agent Injector, que permitan satisfacer los siguientes casos de uso:
1. Certificados de cliente, que ahora se almacenan en un Java KeyStore. En el yml de configuración del repositorio Git guardamos el base64 del JKS y la password del mismo cifrada.
2. Truststores, que ahora se almacenan en un Java KeyStore. En el yml de configuración del repositorio Git guardamos el base64 del JKS y la password del mismo cifrada.
3. Username/password utilizados en Basic HTTP Authentication para WS SOAP y servicios REST.
4. API Keys utilizadas también por clientes REST.
5. Username/password de las bases de datos de Oracle de cada entorno.
6. tpnName/user de las conexiones a Host de cada entorno.
7. Secretos de firma asociados a los clientes Oauth.
8. Claves privadas utilizadas para cifrado de recursos.


## Configuración del Entorno <a class="anchor" id="second-bullet"></a>

In [ ]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')


## Creación de Secretos Estáticos <a class="anchor" id="creación-de-secretos-estáticos"></a> 

In [ ]:
! vault secrets enable -path=static kv-v2

Creamos una engine para secretos estáticos de tipo KVv2

### Binarios (Base64 JKS + Password)<a class="anchor" id="binarios"></a> 

Usamos `keytool` para crear un certificado

In [ ]:
%env PASSWORD=changeit

In [ ]:
%%bash
keytool -genkeypair \
  -alias myapp \
  -keyalg RSA \
  -keysize 2048 \
  -validity 365 \
  -keystore $WORKDIR/myapp-keystore-2.jks \
  -storepass $PASSWORD \
  -keypass $PASSWORD \
  -dname "CN=localhost, OU=Dev, O=MyCompany, L=City, S=State, C=US"

Subimos el secreto a la engine. Para ello:
1. Creamos un path siguiendo la estructura `<engine_path>/subdirectory/secret`
2. Los secretos estáticos en Vault toman la forma key=value. En este caso tenemos dos keys `cert` y `password`
3. El certificado en sí es un objeto binario que para guardarlo en Vault se aconseja que se condifique en base64. Como [referencia](https://support.hashicorp.com/hc/en-us/articles/5332149468691-Storing-pfx-certs-and-binary-files-in-Vault-KV-secrets)
4. El password es información en texto plano

In [ ]:
%%bash
DATA=$(cat $WORKDIR/myapp-keystore-2.jks | base64 | tr -d '\n')
vault kv put static/jks/myapp-keystore-2 cert=$DATA password=$PASSWORD \

Podemos leer la información de vuelta

In [ ]:
! vault kv get -format=json static/jks/myapp-keystore-2 | jq -r .

Tenga en cuenta que el valor del certificado está codificado en base64, de forma que para su consumo se precisa de un paso de decoding previo. En la siguiente celda descargamos el certificado a un fichero y lo leemos utilizando la aplicación `keytool`

In [ ]:
%%bash
vault kv get -format=json static/jks/myapp-keystore-2 | jq -r .data.data.cert | base64 --decode > $WORKDIR/myapp-keystore-2_read.jks

keytool -list -v -keystore $WORKDIR/myapp-keystore-2_read.jks -storepass $PASSWORD

Otra posibilidad es replicando el patrón actual donde tanto el certificado como su correspondiente secreto se encuentran dentro de un fichero de configuración en formato yaml. Construyamos un ejemplo de consumo

In [ ]:
%%bash
cat > ${WORKDIR}/config.yaml <<EOF
---
java:
  application:
    truststore:
      certificate_base64: |
        $(cat $WORKDIR/myapp-keystore-2.jks | base64 | tr -d '\n')
      password: $(echo -n $PASSWORD)

EOF


In [ ]:
%%bash
# Subimos el fichero de configuración a Vault
vault kv put static/jks/config yaml=@$WORKDIR/config.yaml 



In [ ]:
%%bash
# Leemos el secreto de vuelta
vault kv get -format=json static/jks/config | jq -r .data.data.yaml

### ¿Cómo consumimos este secreto desde una aplicación que corre en Kubernetes?

El primer paso es extender la política de acceso para incluir el path al secreto estático que contiene el keystore

In [ ]:
%%bash
vault policy write devk8s - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
path "database/creds/readonly" {
  capabilities = [ "read"]
}
# Para secretos de tipo kv-v2 podemos usar el path del secreto. En este caso damos acceso tanto a los datos como a los metadatos
path "static/+/jks/*" {
    capabilities = ["read"]
}
EOF

El siguiente paso es asegurarse de que la service account que usará el POD/Deployment esté asociada a la política del paso previo

In [ ]:
%%bash

vault write auth/kubernetes/role/role \
    bound_service_account_names=default,usecase \
    bound_service_account_namespaces=test,usecase \
    policies=devk8s \
    ttl=10m

Creamos un namespace en kubernetes denominado `usecase` donde se crearon los recursos que integraran con Vault

In [ ]:
%%bash
kubectl create ns usecase
kubectl create serviceaccount usecase -n usecase

Desplegamos un POD que haga uso de la sa que acabamos de crear

In [ ]:
%%bash
cat > ${WORKDIR}/pod1_usecase.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: pod1
  namespace: usecase
spec:
  serviceAccountName: usecase
  containers:
  - name: pod1
    image: nginx
EOF

kubectl apply -f ${WORKDIR}/pod1_usecase.yaml
sleep 10
kubectl get pods -n usecase

In [ ]:
# En una terminal ejecuta los siguientes comandos
kubectl exec -i -t pod1 -n usecase -- /bin/bash 

# ----------------------

# Instalar jq y curl
apt-get update
apt-get install -y jq curl default-jre


# Configuración de variables de entorno
export VAULT_ADDR=https://vault.vault.svc.cluster.local:8200
export TOKEN=$(cat /var/run/secrets/kubernetes.io/serviceaccount/token)
export VAULT_SKIP_VERIFY=true
export VAULT_K8S_MOUNT_POINT=auth/kubernetes
export SECRET_PATH=static/data/jks/myapp-keystore-2
export SECRET_PATH2=static/data/jks/config

# Autenticación contra Vault usando el token de la service account
export VAULT_TOKEN=$(curl -s -k --request POST --data '{"jwt": "'"$TOKEN"'", "role": "role" }' $VAULT_ADDR/v1/$VAULT_K8S_MOUNT_POINT/login | jq -r .auth.client_token)

# Uso del token de Vault para obtener el secreto
curl -s -k --header "X-Vault-Token: $VAULT_TOKEN" $VAULT_ADDR/v1/$SECRET_PATH | jq -r .data.data > cert

# Get the second secret
curl -s -k --header "X-Vault-Token: $VAULT_TOKEN" $VAULT_ADDR/v1/$SECRET_PATH2 | jq -r .data.data.yaml

# Obtenemos la parte del certificado y lo decodificamos
# y lo guardamos en un archivo keystore.jks
cat cert | jq -r .cert | base64 -d > keystore.jks

# Comprobación de la descarga
keytool -list -v -keystore keystore.jks -storepass $(cat cert | jq -r .password)


Este proceso puede realizarse:
* Directamente a nivel de aplicación haciendo uso de SDKs, con la consiguiente refactorización de la aplicación, por ejemplo con [Spring Cloud Vault](https://cloud.spring.io/spring-cloud-vault/reference/html/#_quick_start)
* Haciendo uso de VSO y sus capacidades de templating
* Haciendo uso de Vault Agent y sus capacidades de templating

En este caso, nos focalizamos en las dos últimas alternativas

### VSO con templating

Creamos un nuevo VaultAuth para permitir el acceso desde el namespace `usecase`

In [ ]:
%%bash
cat > ${WORKDIR}/vaultauth_crd_usecase1.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  namespace: vault-secrets-operator
  name: vaultauth
spec:
  vaultConnectionRef: example
  allowedNamespaces: ["usecase"]
  method: kubernetes
  mount: kubernetes

  kubernetes:
    # role to use when authenticating to Vault
    role: role
    serviceAccount: usecase

EOF
kubectl apply -f ${WORKDIR}/vaultauth_crd_usecase1.yaml

Creamos un nuevo `VaultStaticSecret` incluyendo el `transformation` attribute

In [ ]:
%%bash

cat > ${WORKDIR}/vso_usecase1.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  name: myapp-keystore-2
  namespace: usecase
spec:
  vaultAuthRef: vault-secrets-operator/vaultauth
  mount: static
  type: kv-v2
  path: jks/myapp-keystore-2
  refreshAfter: 60s
  destination:
    create: true
    name: myapp-keystore-2
    transformation:
      excludes:
        - ".*"
      templates:
        cert:
          text: |
            {{-  printf "%s" (get .Secrets "cert") | b64dec -}}
        password:
          text: |
            {{- printf "%s" (get .Secrets "password") -}}
EOF

kubectl apply -f ${WORKDIR}/vso_usecase1.yaml

Para el caso del fichero yaml, no hace falta que hagamos ninguna transformación

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > ${WORKDIR}/vso_usecase2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  name: myapp-keystore-2
  namespace: usecase
spec:
  vaultAuthRef: vault-secrets-operator/vaultauth
  mount: static
  type: kv-v2
  path: jks/config
  refreshAfter: 60s
  destination:
    create: true
    name: config-yaml

EOF

kubectl apply -f ${WORKDIR}/vso_usecase2.yaml

Leemos el secreto de vuelta

In [ ]:
%%bash
echo -n "$(kubectl get secret config-yaml -n usecase -o json | jq -r .data.yaml | base64 -d)"

Ahora desplegamos un POD donde montamos el secreto

In [ ]:
%%bash
cat > ${WORKDIR}/vso__pod_usecase.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: vso-myapp-keystore
  namespace: usecase
spec:
  containers:
  - name: vso-myapp-keystore
    image: redis
    volumeMounts:
    - name: keystore
      mountPath: "/etc/keystore"
      readOnly: true
  volumes:
  - name: keystore
    secret:
      secretName: config-yaml
      optional: true
EOF

kubectl apply -f ${WORKDIR}/vso__pod_usecase.yaml
sleep 10

In [ ]:
%%bash
# Chequea secretos
kubectl exec vso-myapp-keystore -n usecase  -- ls -l /etc/keystore
echo ""
echo "------"
kubectl exec vso-myapp-keystore -n usecase -- cat /etc/keystore/yaml

### Vault Agent con templating

In [ ]:
%%bash
cat > $WORKDIR/agent-myapp-keystore-1.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: agent-myapp-keystore-1
  namespace: usecase
  labels:
    app: agent-myapp-keystore-1
spec:
  selector:
    matchLabels:
      app: agent-myapp-keystore-1
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/role: 'role'
        vault.hashicorp.com/tls-skip-verify: 'true' # Untrusted cert used here
        vault.hashicorp.com/agent-inject-template-keystore.jks: |
          {{- with secret "static/data/jks/myapp-keystore-2" -}}
            {{ base64Decode .Data.data.cert }}
          {{- end -}}
        vault.hashicorp.com/agent-inject-template-password: |
          {{- with secret "static/data/jks/myapp-keystore-2" -}}
            {{ .Data.data.password }}
          {{- end -}}
      labels:
        app: agent-myapp-keystore-1
    spec:
      serviceAccountName: usecase
      containers:
      - name: agent-myapp-keystore-1
        image: nginx
EOF

Desplegamos el POD

In [ ]:
! kubectl apply --filename $WORKDIR/agent-myapp-keystore-1.yaml

Vault Agent Inject despliega un init container así como un sidecar container. El init container se encarga de descargar el secreto y el sidecar lo inyecta en el contenedor principal.
> The `init` container will prepopulate the shared memory volume with the requested secrets prior to the other containers starting. The `sidecar` container will continue to authenticate and render secrets to the same location as the pod runs. Using annotations, the initialization and sidecar containers may be disabled.

Chequeamos los logs del init container

In [ ]:
%%bash 
sleep 10
kubectl logs -n usecase\
    $(kubectl get pod -n usecase -l app=agent-myapp-keystore-1 -o jsonpath="{.items[0].metadata.name}") \
    --container vault-agent

Verificamos que las dos keys presentes en el secreto se han renderizado en sus respectivos ficheros dentro del path `/vault/secrets`

In [ ]:
%%bash
kubectl exec -n usecase\
    $(kubectl get pod -n usecase -l app=agent-myapp-keystore-1 -o jsonpath="{.items[0].metadata.name}") \
    --container agent-myapp-keystore-1 -- ls /vault/secrets

keysotre.jks contiene el certificado en formato jks (binario)

In [ ]:
%%bash
kubectl exec -n usecase\
    $(kubectl get pod -n usecase -l app=agent-myapp-keystore-1 -o jsonpath="{.items[0].metadata.name}") \
    --container agent-myapp-keystore-1 -- cat /vault/secrets/keystore.jks

password contiene el password del certificado, en este caso en texto plano

In [ ]:
%%bash
kubectl exec -n usecase\
    $(kubectl get pod -n usecase -l app=agent-myapp-keystore-1 -o jsonpath="{.items[0].metadata.name}") \
    --container agent-myapp-keystore-1 -- cat /vault/secrets/password

### Secretos como Texto <a class="anchor" id="secretos-como-texto"></a>

In [ ]:
%%bash
# API keys (usamos https://codepen.io/corenominal/pen/rxOmMJ para la generación de una API key de ejemplo )
API_KEY="0796af05-5a14-4aa3-9494-24652f3f3e92"

# Guardamos la API key dentro del secreto static/api/key-1
# En este caso no es necesario el uso de base64 ya que el valor es un string
# y no un binario
vault kv put static/api/key-1 key=$API_KEY

In [ ]:
# Leemos el secreto de vuelta
# Los metadatos incorporan la versión del secreto y fecha de creación
! vault kv get -format=json static/api/key-1 | jq -r .

In [ ]:
# Podemos obtener más metadatos de vuelta usando  la opción `metadata`
# Que permite ver no sólo cuando se creo el secreto sino también la fecha de última modificación, creación por versiones, número de versiones, estado de softdelete, etc
! vault kv metadata get -format=json static/api/key-1 | jq -r .

In [ ]:
# Añadimos custom metadata al secreto
! vault kv metadata put -custom-metadata=env="dev" -custom-metadata=owner="devops" \
    -custom-metadata=team="devops" \
    -custom-metadata=app="myapp" \
    -custom-metadata=version="1.0" \
    -custom-metadata=description="API key for myapp" \
    static/api/key-1

In [ ]:
! vault kv metadata get -format=json static/api/key-1 | jq -r .

Para montar el secreto en el POD usaremos tanto VSO como Vault Agent Injector. Los pasos a seguir serán similares al caso previo:
* Crear una política que de permisos de lectura al path del secreto (tanto datos como metadatos)
* Asociar policy a un K8S Auth Role. Este role asociará la SA de Kubernetes al role.
* VSO:
  * Creación de CRD para sincronizar el secreto a secreto de kubernetes.
  * Creación de POD y montamos el secreto como variable de entorno
* Vault Agent Injector
  * Creación de POD con anotaciones para montar el secreto como fichero.

In [ ]:
%%bash
vault policy write api-key-policy - <<EOF
# Para secretos de tipo kv-v2 podemos usar el path del secreto. En este caso damos acceso tanto a los datos como a los metadatos
path "static/+/api/*" {
    capabilities = ["read"]
}
EOF

In [ ]:
%%bash
# Creamos un role para el acceso a la API
vault write auth/kubernetes/role/apirole \
    bound_service_account_names=apisa \
    bound_service_account_namespaces=usecase \
    policies=api-key-policy \
    ttl=10m

In [ ]:
%%bash
# Creamos el service account
kubectl create serviceaccount apisa -n usecase

In [ ]:
%%bash
# Creamos el VaultAuth CRD para dar acceso a la API key usando la apisa service account
cat > ${WORKDIR}/vaultauth_api_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  namespace: vault-secrets-operator
  name: apirole
spec:
  vaultConnectionRef: example
  allowedNamespaces: ["usecase"]
  method: kubernetes
  mount: kubernetes

  kubernetes:
    # role to use when authenticating to Vault
    role: apirole
    serviceAccount: apisa

EOF
kubectl apply -f ${WORKDIR}/vaultauth_api_crd.yaml

In [ ]:
%%bash
## Support KVv1 and KVv2
cat > ${WORKDIR}/static_secret_api.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: usecase
  name: api-key-1
spec:
  vaultAuthRef: vault-secrets-operator/apirole # Usamos el VaultAuth objeto creado antes
  mount: static
  type: kv-v2
  path: api/key-1
  refreshAfter: 60s
  destination:
    create: true
    name: api-key-1
EOF

kubectl apply -f ${WORKDIR}/static_secret_api.yaml

In [ ]:
%%bash
cat > ${WORKDIR}/apipod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: apipod
  namespace: usecase
spec:
  # Usamos la apisa service account
  serviceAccountName: apisa
  containers:
  - name: apipod
    image: nginx
    env:
    - name: API_KEY
      valueFrom:
        secretKeyRef:
          name: api-key-1
          key: key
EOF

kubectl apply -f ${WORKDIR}/apipod.yaml
sleep 10



In [ ]:
%%bash
# Verificamos que el secreto se ha inyectado correctamente como variable de entorno 
kubectl exec -n usecase apipod -- env | grep API_KEY

El ejercicio equivalente con Vault Agent inyector podría realizarse de la siguiente forma

In [ ]:
%%bash
cat > $WORKDIR/agent-api-key-1.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: agent-api-key-1
  namespace: usecase
  labels:
    app: agent-api-key-1
spec:
  selector:
    matchLabels:
      app: agent-api-key-1
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/role: 'apirole'
        vault.hashicorp.com/tls-skip-verify: 'true' # Untrusted cert used here
        # Seleccionamos el secreto de la API key
        vault.hashicorp.com/agent-inject-secret-api-key: "static/data/api/key-1"
        # Renderizamos exclusivamente el valor de la API key
        vault.hashicorp.com/agent-inject-template-api-key: |
          {{- with secret "static/data/api/key-1" -}}
            {{ .Data.data.key }}
          {{- end -}}
      labels:
        app: agent-api-key-1
    spec:
      serviceAccountName: apisa
      containers:
      - name: agent-api-key-1
        image: nginx
EOF

In [ ]:
! kubectl apply --filename $WORKDIR/agent-api-key-1.yaml

In [ ]:
%%bash
# Esperamos a que el pod esté en estado Running
sleep 10
# Verificamos que la inyección del secreto ha funcionado
kubectl exec -n usecase\
    $(kubectl get pod -n usecase -l app=agent-api-key-1 -o jsonpath="{.items[0].metadata.name}") \
    --container agent-api-key-1 -- cat /vault/secrets/api-key

# Almacenando claves de criptografía <a class="anchor" id="crypto"></a>

In [ ]:
%%bash
# Generamos un par de claves RSA
# Clave privada
openssl genrsa -out $WORKDIR/private.pem 2048
# Clave pública
openssl rsa -in $WORKDIR/private.pem -pubout -out $WORKDIR/public.pem

In [ ]:
%%bash
# Guardamos las claves privada y pública en Vault
vault kv put static/oauth/client-1 \
    private_key="$(cat $WORKDIR/private.pem)" \
    public_key="$(cat $WORKDIR/public.pem)"

In [ ]:
# Leemos el secreto de vuelta
! vault kv get -format=json static/oauth/client-1 | jq -r .


In [ ]:
%%bash
# Creamos una policy para el acceso a los secretos de OAuth
vault policy write oauth-policy - <<EOF 
# Para secretos de tipo kv-v2 podemos usar el path del secreto. En este caso damos acceso tanto a los datos como a los metadatos
path "static/+/oauth/*" {
    capabilities = ["read"]
}
EOF

In [ ]:
%%bash
# Creamos el role para el acceso a los secretos
vault write auth/kubernetes/role/oauth-role \
    bound_service_account_names=oauthsa \
    bound_service_account_namespaces=usecase \
    policies=oauth-policy \
    ttl=10m

In [ ]:
# Creamos el service account
! kubectl create serviceaccount oauthsa -n usecase

### VSO

VaultAuth para el role oauth-role y service account oauthsa

In [ ]:
%%bash
# Creamos el VaultAuth CRD para dar acceso a la API key usando la apisa service account
cat > ${WORKDIR}/vaultauth_oauth_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultAuth
metadata:
  namespace: vault-secrets-operator
  name: oauth-role
spec:
  vaultConnectionRef: example
  allowedNamespaces: ["usecase"]
  method: kubernetes
  mount: kubernetes

  kubernetes:
    # role to use when authenticating to Vault
    role: oauth-role
    serviceAccount: oauthsa

EOF
kubectl apply -f ${WORKDIR}/vaultauth_oauth_crd.yaml

Sincronizamos el secreto usando el VaultAuth previo

In [ ]:
%%bash

cat > ${WORKDIR}/oauth_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1beta1
kind: VaultStaticSecret
metadata:
  namespace: usecase
  name: oauth
spec:
  vaultAuthRef: vault-secrets-operator/oauth-role
  mount: static
  type: kv-v2
  # Usamos el path del secreto
  path: oauth/client-1
  destination:
    create: true
    name: oauth
EOF

kubectl apply -f ${WORKDIR}/oauth_secret.yaml


Montamos los secretos en un POD

In [ ]:
%%bash
cat > ${WORKDIR}/pod_oauth.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: pod-oauth
  namespace: usecase
spec:
  containers:
  - name: pod-oauth
    image: redis
    volumeMounts:
    - name: pod-oauth
      mountPath: "/etc/oauth"
      readOnly: true
  volumes:
  - name: pod-oauth
    secret:
      secretName: oauth
      optional: true
EOF

# Despliega el POD
kubectl apply -f ${WORKDIR}/pod_oauth.yaml

Chequeamos el secreto

In [ ]:
%%bash
# Espera a que despliegue
sleep 10 
# Chequea secretos
kubectl exec pod-oauth -n usecase  -- ls /etc/oauth/
echo ""
echo "------"
kubectl exec pod-oauth -n usecase -- cat /etc/oauth/private_key
echo "------"
kubectl exec pod-oauth -n usecase -- cat /etc/oauth/public_key

### Vault Agent

In [ ]:
%%bash
cat > $WORKDIR/agent-oauth-1.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: oauth-client
  namespace: usecase
  labels:
    app: oauth-client
spec:
  selector:
    matchLabels:
      app: oauth-client
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/role: "oauth-role"
        vault.hashicorp.com/tls-skip-verify: 'true' # Untrusted cert used here
        vault.hashicorp.com/agent-inject-template-private-key: |
          {{- with secret "static/data/oauth/client-1" -}}
          {{ .Data.data.private_key }}
          {{- end -}}
        vault.hashicorp.com/agent-inject-template-public-key: |
          {{- with secret "static/data/oauth/client-1" -}}
          {{ .Data.data.public_key }}
          {{- end -}}
      labels:
        app: oauth-client
    spec:
      serviceAccountName: oauthsa
      containers:
      - name: oauth-client
        image: nginx
EOF

In [ ]:
%%bash
kubectl apply --filename $WORKDIR/agent-oauth-1.yaml
sleep 10

In [ ]:
%%bash
kubectl exec -n usecase \
    $(kubectl get pod -n usecase -l app=oauth-client -o jsonpath="{.items[0].metadata.name}") \
    -- cat /vault/secrets/private-key

kubectl exec -n usecase \
    $(kubectl get pod -n usecase -l app=oauth-client -o jsonpath="{.items[0].metadata.name}") \
    -- cat /vault/secrets/public-key

# Operaciones de Criptografía usando la Transit Secret Engine <a class="anchor" id="transit"></a>

In [ ]:
! vault secrets enable transit

In [ ]:
! vault write -f transit/keys/orders exportable=true

### Encrypt/Decrypt

In [ ]:
%%bash
# ENCRYPT
vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext

export CIPHERTEXT=$(vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext)

# DECRYPT
vault write transit/decrypt/orders ciphertext=$CIPHERTEXT -format=json | jq -r .data.plaintext | base64 -d

In [ ]:
%%bash
# update encryption key
vault write -f transit/keys/orders/rotate 

In [ ]:
%%bash
export CIPHERTEXT=$(vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext)
echo $CIPHERTEXT
# DECRYPT
vault write transit/decrypt/orders ciphertext=$CIPHERTEXT -format=json | jq -r .data.plaintext | base64 -d

In [ ]:
%%bash
# update encryption key
vault write -f transit/keys/orders/rotate 

In [ ]:
%%bash
# Set minimum decryption version
vault write transit/keys/orders/config min_decryption_version=3 

Intentamos descifran un cyphertext creado usando la versión 2, como hemos definido la versión 3 como  la última a utilizar para desencriptar, la función nos devuelve un error

In [ ]:
%%bash
# DECRYPT
export CIPHERTEXT="vault:v2:9WWxS1FxVF+pOmmCc5asxFYqDGJ4B+Cly2mMI2Rpi5MkUC9BuJPtmNduoMOucGAU"
vault write transit/decrypt/orders ciphertext=$CIPHERTEXT -format=json | jq -r .data.plaintext | base64 -d

### Signing

In [ ]:
! vault write -f  transit/keys/signature type=rsa-4096

In [ ]:
%%bash
INPUT=$(echo -n "un string no muy largo" | base64)
# Operación de firma
SIGNATURE=$(vault write -format=json transit/sign/signature input=$INPUT hash_algorithm=sha2-256 -format=json | jq -r .data.signature)
echo $SIGNATURE

# Verificación de firma
vault write transit/verify/signature input=$INPUT signature=$SIGNATURE  hash_algorithm=sha2-256